In [9]:
import requests
from bs4 import BeautifulSoup
import re
import urllib.request
import cv2
import matplotlib.pyplot as plt
import os

## ÍNDICE

### 1. Clasificación de la Liga
### 2. Estadísticas de Árbitros
### 3. Enfrentamiento Histórico entre equipos
### 4. Jugadores más destacados
### 5. Asistencia media al campo de un equipo
### 6. Resultados de un equipo según el árbitro
### 7. Balance de goles por temporada
### 8. PROGRAMA PRINCIPAL

# 1. Clasificación de la Liga

In [2]:
def clasificacion():
    temps = [f'200{x}-0{x+1}' for x in range(1,9)] + [f'201{x}-1{x+1}' for x in range(0,9)] + ['2019-20'] + ['2020-21']
    temp = preguntar(temps)
    link = f'https://es.wikipedia.org/wiki/Primera_Divisi%C3%B3n_de_Espa%C3%B1a_{temps[temp]}'
    src  = requests.get(link)
    soup = BeautifulSoup(src.content, "html") #obtener el src de la temporada
    tabla = soup.find('table', align="center", cellpadding="3", cellspacing="0")
    filas = tabla.find_all('tr')
    for fila in filas:
        fila = fila.text.strip().splitlines()[0:5:2]
        print(fila[0],'|', fila[1],'|',fila[2])

# 2. Estadísticas de Árbitros

In [2]:
def arbfunc(ini,fin):
        for palabra in fin:
            if palabra not in ini:
                return False
        return True

    
def arbitros():    
    temporada=int(input('¡Que Temporada quieres analizar? (Entre 2016 y 2020):'))
    temp=0
    if temporada==2020:
        temp=1
    url=f'https://es.wikipedia.org/wiki/Primera_Divisi%C3%B3n_de_Espa%C3%B1a_{temporada}-{temporada-1999}'
    listas=['font-size:95%; text-align: center; width: 100%;',"font-size:95%; text-align: center; width: 100%; margin:0.5em auto 0.5em auto;"]
    result=requests.get(url)

    src=result.content

    soup=BeautifulSoup(src,'lxml')

    arb=soup.find('div',class_='columns')
    arb2=arb.find_all('li')
    arbf=[]
    for arbitro in arb2:
        a=arbitro.find('small')
        b=a.find('a')
        arbf.append(b.text)

    for i,arb in enumerate(arbf):
        print(i,')',arb)
    ar=int(input('Arbitro :'))
    arb=arbf[ar]  
    liy=arb.split()


    partidos2=[]
    partidos3=[]

    jornadas=soup.find_all('table',style=listas[temp])
    for jornada in jornadas:
        partidos=[]
        part=jornada.find_all('tr')
        if len(part)==12:
            partidos2.append(part.copy())
    for jornada in partidos2:
        for i in range(2):
            del jornada[0]
        for partido in jornada:
            datos=partido.find_all('td')
            listadatos=[]
            for dato in datos:
                listadatos.append(dato.text)
            partidos3.append(listadatos)
    for partido in partidos3:
        del partido[3]
        for i in range(3):
            if len(partido)>6:
                del partido[3]

    am=[]
    rj=[]
    for partido in partidos3:
        a=partido[3]
        if arbfunc(liy,a.split()):
            am.append(partido[4])
            rj.append(partido[5])

    mam=sum(int(e) for e in am)/len(am)
    mrj=sum(int(e) for e in rj)

    print('Media Amarillas: ',mam)

    print('Número Expulsiones: ',mrj)

    print('Máximo Amarillas: ',max(am))

# 3. Enfrentamiento Histórico entre equipos

In [5]:
#44 temporadas
#10 funciones

def ebien(e_mod):
    ebien = ''
    for x in e_mod:
        ebien += (x+' ')
    return ebien



def clear(): #Limpiar la pantalla
    if os.name == "nt":
        os.system("cls")
    else:
        os.system("clear")



def sopa(link): #Dado un link saca el soup
    src = requests.get(link)
    soup = BeautifulSoup(src.content, "html")
    return soup



def imagen(algo):
    url='https://www.futbol-regional.es/'+algo

    imagen= open('imgdesc.jpg','wb')
    imagen.write(urllib.request.urlopen(url).read())
    imagen.close()

    imgmal=cv2.imread('imgdesc.jpg')
    imgbien=cv2.cvtColor(imgmal,cv2.COLOR_BGR2RGB)
    
    plt.xticks([])
    plt.yticks([])
    plt.imshow(imgbien)
    plt.show()

    
    
def mostrar(e1_mod, e2_mod): #Devuelve los semilinks de las págs de cada equipo
    link = 'https://www.futbol-regional.es/escudos.php'
    src = requests.get(link)
    soup = BeautifulSoup(src.content, "html")
    equipos = soup.find_all('div', class_='columna_escudo')
    img1 = None
    img2 = None
    for escudo in equipos:
        if img1 != None and img2 != None:
            break
        i = escudo.find('img') #<img align="absmiddle" src="img/escudos_140/30744.jpg"/>
        if i != None: #Es un escudo, no el título de una liga
            texto = escudo.find_all('a')
            nom = re.findall(r'\b[\.]?(\w+[^\.]+)[\.]?\b', texto[1].text)[0]
            if 'Espanyol' in nom:
                nom = 'Espanyol'
            if 'Rayo Vallecano' in nom:
                nom = 'Rayo Vallecano'
            if 'Athletic' in nom:
                nom = 'Athletic'
            if nom in e1_mod or e1_mod in nom:
                img1 = re.findall(r'src="(.+)"', str(i))[0]
            if nom in e2_mod or e2_mod in nom:
                img2 = re.findall(r'src="(.+)"', str(i))[0]
    return (img1, img2)



def links_equipos(temp): #Saca la lista de equipos de una temporada
    equipos = []
    link = f'https://es.wikipedia.org/wiki/Primera_Divisi%C3%B3n_de_Espa%C3%B1a_{temp}'
    src = requests.get(link)
    soup = BeautifulSoup(src.content, "html")
    tablas = soup.find_all('table', class_='sortable')
    for x in tablas:
        y = x.find('tr')
        z = y.find('th')
        if 'Equipo' in z.text:
            tabla = x
            break
    filas = tabla.find_all('tr')
    for fila in range(1, len(filas)):
        celdas = filas[fila].find_all('td')
        try:
            if temp in ['1995-96', '1990-91', '2014-15']:
                casillas = celdas[0].find_all('a')
                equipos.append(casillas[1].attrs['href'])
            else:
                equipos.append(celdas[0].a.attrs['href'])
        except IndexError:
            pass
    return equipos
    
    
def equipos(temp): #Saca la lista de equipos de una temporada
    equipos = []
    link = f'https://es.wikipedia.org/wiki/Primera_Divisi%C3%B3n_de_Espa%C3%B1a_{temp}'
    src = requests.get(link)
    soup = BeautifulSoup(src.content, "html")
    tablas = soup.find_all('table', class_='sortable')
    for x in tablas:
        y = x.find('tr')
        z = y.find('th')
        if 'Equipo' in z.text:
            tabla = x
            break
    filas = tabla.find_all('tr')
    for fila in range(1, len(filas)):
        celdas = filas[fila].find_all('td')
        try:
            equipos.append(celdas[0].text.splitlines()[0])
        except IndexError:
            pass
    return equipos



def resultados_cruzados(tabla, r1, r2): #Ath - back - res #back - res #Ath - res
    a = False
    vict_e1 = 0 
    vict_e2 = 0 
    empates = 0
    filas = tabla.find_all('tr')
    for x in [r1, r2]:
        resuls = filas[x+1].find_all('td')#filas[r1+1]
        celda0 = filas[1].find_all('td')[0].text
        celda1 = filas[2].find_all('td')[0].text
        if (len(celda0)>4) and (len(celda1)>4): 
            texto = resuls[r2+1].text
        else:
            texto = resuls[r2].text
        nums = re.findall(r'[0-9]+', texto)
        if int(nums[0]) > int(nums[1]):
            if a == False:
                vict_e1 += 1
            else:
                vict_e2 += 1
        elif int(nums[0]) < int(nums[1]):
            if a == False:
                vict_e2 += 1
            else:
                vict_e1 += 1
        else:
            empates += 1
        aux = r1
        r1 = r2
        r2 = aux
        a = True
    return (vict_e1, vict_e2, empates)



def resultados_jornadas(soup, link_teams, prov_e1, prov_e2,  tipo): #link_teams, limk_e1, link_e2
    vict_e1 = 0 
    vict_e2 = 0
    empates = 0
    a = False
    if tipo == "wikitable mw-collapsible mw-collapsed":
        tabla = soup.find_all('table', class_={tipo})[:38]
    elif tipo == "wikitable collapsible collapsed":
        tabla = soup.find_all('table', class_={tipo})
    for partido in tabla[:19]: #Para cada una de las 19 primeras jornadas
        if a == True:
            break
        filas = partido.find_all('tr')
        cuadro = filas[2:]
        if len(filas) == 230:
            cuadro = filas[2:]
            jornadas = filas[2::12]
            col = filas[3::12]
            for x in [jornadas, col]:
                for y in x:
                    cuadro.remove(y)
        for fila in cuadro:
            if a == True:
                break
            lista = re.findall(r'\w.+', fila.text)
            algo = fila.find_all('td')
            uno = algo[0].a.attrs['href']
            dos = algo[2].a.attrs['href']
            if (prov_e1 in uno or prov_e1 in dos) and (prov_e2 in uno or prov_e2 in dos):
                nums = re.findall(r'[0-9]+', lista[1])
                if int(nums[0]) > int(nums[1]):
                    if prov_e1 in uno:
                        vict_e1 += 1
                    else:
                        vict_e2 += 1
                elif int(nums[0]) < int(nums[1]):
                    if prov_e1 in uno:
                        vict_e2 += 1
                    else:
                        vict_e1 += 1
                else:
                    empates += 1
                a = True #Cuando encuentra el partido entre e1 y e2, se corta para no hacer cálculos innecesarios
    a = False
    
    vuelta2 = tabla[19:]
    partido = tabla[0]
    filas = partido.find_all('tr')
    if len(filas) == 230:
        vuelta2 = tabla[20:]
    for partido in vuelta2: #Para cada una de las 19 últimas jornadas
        if a == True:
            break
        filas = partido.find_all('tr')
        cuadro = filas[2:]
        if len(filas) == 230:
            cuadro = filas[2:]
            jornadas = filas[2::12]
            col = filas[3::12]
            for x in [jornadas, col]:
                for y in x:
                    cuadro.remove(y)
        for fila in cuadro:
            if a == True:
                break
            lista = re.findall(r'\w.+', fila.text)
            algo = fila.find_all('td')
            uno = algo[0].a.attrs['href']
            dos = algo[2].a.attrs['href']
            if (prov_e1 in uno or prov_e1 in dos) and (prov_e2 in uno or prov_e2 in dos):
                nums = re.findall(r'[0-9]+', lista[1])
                if int(nums[0]) > int(nums[1]):
                    if prov_e1 in uno:
                        vict_e1 += 1
                    else:
                        vict_e2 += 1
                elif int(nums[0]) < int(nums[1]):
                    if prov_e1 in uno:
                        vict_e2 += 1
                    else:
                        vict_e1 += 1
                else:
                    empates += 1
                a = True #Cuando encuentra el partido entre e1 y e2, se corta para no hacer cálculos innecesarios
    a = False
    return (vict_e1, vict_e2, empates)




def historico(): 
    temps = [f'19{x}-{x+1}' for x in range(76,99)] + ['1999-2000'] + [f'200{x}-0{x+1}' for x in range(0,9)] + [f'201{x}-1{x+1}' for x in range(0,9)] + ['2019-20'] + ['2020-21']
    temp = preguntar(temps)
    links = links_equipos(temps[temp])
    teams = equipos(temps[temp])
    print('COD    EQUIPO')
    for ind,equipo in enumerate(teams): #El usuario ha de ver las opciones
        print(ind,'-->',equipo)
    r1 = 0
    r2 = 0
    while r1 == r2:
        try:
            r1 = int(input('COD equipo 1:'))
            r2 = int(input('COD equipo 2:'))
            if r1 == r2:
                print('Has de escoger dos equipos distintos')
            if r1 < 0 or r1 > 19 or r2 < 0 or r2 > 19:
                print('Alguno de los COD que has seleccionado no pertenece a ningún equipo')
                r1 = 0
                r2 = 0
            e1 = teams[r1]
            e2 = teams[r2]
        except ValueError:
            print('Error! Tienes que escribir un número')
            r1 = 0
            r2 = 0
        except IndexError:
            print('Vuelve a escribirlos')
            r1 = 0
            r2 = 0
    print('Tus equipos son:', e1,'y', e2)
    vict_e1 = 0
    vict_e2 = 0
    empates = 0
    link_e1 = links[teams.index(e1)]
    link_e2 = links[teams.index(e2)]
    print('Comprobando temporadas...')
    for x in temps: #lista con los sufijos de cada teporada
        print(x)
        links_teams = links_equipos(x) #Links de temporada x
        if link_e1 in links_teams and link_e2 in links_teams:  #Ver si ambos equipos están en la liga. En caso positivo:
            i1 = links_teams.index(link_e1) #indice equipo1 seleccionado 
            i2 = links_teams.index(link_e2) #indice equipo2 seleccionado
            prov_e1 = links_teams[i1]
            prov_e2 = links_teams[i2]
            link = f'https://es.wikipedia.org/wiki/Primera_Divisi%C3%B3n_de_Espa%C3%B1a_{x}'
            src = requests.get(link)
            soup = BeautifulSoup(src.content, "html") #obtener el src de la temporada
            tabla = soup.find_all('table', border="2")
            if x in ['2006-07','2007-08']:
                    tablas = soup.find_all('table', class_='wikitable')
                    a, b, c = resultados_cruzados(tablas[1], i1, i2)
                    vict_e1 += a
                    vict_e2 += b
                    empates += c
            elif x in ['1976-77', '1977-78', '1979-80', '1980-81', '1981-82', '1986-87', '1999-2000', '2008-09']:
                    tabla = soup.find('table', border="2")
                    a, b, c = resultados_cruzados(tabla, i1, i2)
                    vict_e1 += a
                    vict_e2 += b
                    empates += c
            elif len(tabla) == 1:
                    tabla = soup.find('table', border="2")
                    a, b, c = resultados_cruzados(tabla, i1, i2)
                    vict_e1 += a
                    vict_e2 += b
                    empates += c
            elif len(soup.find_all('table', class_="wikitable collapsible collapsed")) > 30:
                    tipo = "wikitable collapsible collapsed"
                    a, b, c = resultados_jornadas(soup, links_teams, prov_e1, prov_e2,  tipo)
                    vict_e1 += a
                    vict_e2 += b
                    empates += c
            elif x == '2016-17':
                    tipo = "wikitable mw-collapsible mw-collapsed"
                    a, b, c = resultados_jornadas(soup, links_teams, prov_e1, prov_e2,  tipo)
                    vict_e1 += a
                    vict_e2 += b
                    empates += c
            elif len(soup.find_all('table', class_="wikitable mw-collapsible mw-collapsed")) > 30:
                    tipo = "wikitable mw-collapsible mw-collapsed"
                    a, b, c = resultados_jornadas(soup, links_teams, prov_e1, prov_e2,  tipo)
                    vict_e1 += a
                    vict_e2 += b
                    empates += c
    
    e1_mod = re.findall(r'\b[\.]?(\w{2,20})\s?', e1)
    e2_mod = re.findall(r'\b[\.]?(\w{2,20})\s?', e2)
    e1_mod = ebien(e1_mod)
    e2_mod = ebien(e2_mod)
    img1, img2 = mostrar(e1_mod, e2_mod)  
    imagen(img1)   
    imagen(img2)    
    print(f'Se han jugado {vict_e1+vict_e2+empates} partidos entre {e1} y {e2}')
    print(f'{vict_e1} victorias para el {e1}\n{vict_e2} victorias para {e2}\n{empates} Empates')

# 4. Jugadores más destacados

In [8]:
def img_wiki(url):
    result=requests.get(url)
    src=result.content

    soup=BeautifulSoup(src,'lxml')
    tabla=soup.find('table',class_='infobox')
    tab2=tabla.find_all('td',colspan='3',style='text-align:center;font-size: 92%;;')
    url2=tab2[0].find('img').get('src')
    url='https:'+url2

    imagen= open('imgdesc.jpg','wb')
    imagen.write(urllib.request.urlopen(url).read())
    imagen.close()

    imgmal=cv2.imread('imgdesc.jpg')
    imgbien=cv2.cvtColor(imgmal,cv2.COLOR_BGR2RGB)

    plt.xticks([])
    plt.yticks([])
    plt.imshow(imgbien)
    plt.show()

def img_AS(url):
    imagen= open('imgdesc.jpg','wb')
    imagen.write(urllib.request.urlopen(url).read())
    imagen.close()

    imgmal=cv2.imread('imgdesc.jpg')
    imgbien=cv2.cvtColor(imgmal,cv2.COLOR_BGR2RGB)

    plt.xticks([])
    plt.yticks([])
    plt.imshow(imgbien)
    plt.show()

def jugadores_destacados():
    temporada = {'10':'19-20', '11':'20-21'}
    menu = input('Temporadas: \n 1. 2010-11 \n 2. 2011-12 \n 3. 2012-13 \n 4. 2013-14 \n 5. 2014-15 \n 6. 2015-16 \n 7. 2016-17 \n 8. 2017-18 \n 9. 2018-19 \n 10. 2019-20 \n 11. 2020-21 \n 0. Salir \n\n Opción: ')

    while menu != '0':
        if all(menu != str(i) for i in range(0,12)):
            print(' Error. Por favor, elija una opción adecuada.')
        elif menu == '1':    
            url = 'https://es.wikipedia.org/wiki/Primera_Divisi%C3%B3n_de_Espa%C3%B1a_2010-11'
            result = requests.get(url)

            src = result.content
            soup = BeautifulSoup(src, 'html.parser')

            resultado = []

            aux = []
            tables = soup.find_all('table', class_='wikitable')
            for table in tables:
                if 'Goles' in table.text.strip() or 'Asistencias' in table.text.strip():
                    aux.append(table)

            tablas = [aux[0],aux[1],aux[4]]

            filas = tablas[0].find_all('tr')
            nombre = filas[1].find_all('td')[0].text.strip()
            img = filas[1].find_all('td')[0].find_all('a')[1].attrs['href']
            enlace = 'https://es.wikipedia.org' + img
            numero = int(filas[1].find_all('td')[2].text.strip())
            resultado.append((nombre,[numero,enlace]))

            filas = tablas[1].find_all('tr')
            nombre = filas[1].find_all('td')[1].text.strip()
            img = filas[1].find_all('td')[1].find_all('a')[1].attrs['href']
            enlace = 'https://es.wikipedia.org' + img
            numero = int(filas[1].find_all('td')[3].text.strip())
            resultado.append((nombre,[numero,enlace]))

            filas = tablas[2].find_all('tr')
            nombre = filas[1].find_all('td')[0].text.strip()
            img = filas[1].find_all('td')[0].find_all('a')[1].attrs['href']
            enlace = 'https://es.wikipedia.org' + img
            numero = int(filas[1].find_all('td')[1].text.strip())
            resultado.append((nombre,[numero,enlace]))

            cont = 0

            print('\n')
            print('RESULTADOS')
            print('---------------------------------------------------------')  
            for i in range(len(resultado)):
                if cont == 0:
                    k,v = resultado[i]
                    cont += 1
                    print(f'Máximo goleador: {k}')
                    print(f'Goles: {v[0]}')
                    img_wiki(v[1])
                    print('---------------------------------------------------------')
                elif cont == 1:
                    k,v = resultado[i]
                    cont += 1
                    print(f'Máximo asistente: {k}')
                    print(f'Asistencias: {v[0]}')
                    img_wiki(v[1])
                    print('---------------------------------------------------------')
                else:
                    k,v = resultado[i]
                    print(f'Mejor portero: {k}')
                    print(f'Goles encajados: {v[0]}')
                    img_wiki(v[1])
                    print('---------------------------------------------------------')

        elif menu == '2':
            url = 'https://es.wikipedia.org/wiki/Primera_Divisi%C3%B3n_de_Espa%C3%B1a_2011-12'
            result = requests.get(url)

            src = result.content
            soup = BeautifulSoup(src, 'html.parser')

            resultado = []

            aux = []
            tables = soup.find_all('table', class_='wikitable')
            for table in tables:
                if 'Goles' in table.text.strip() or 'Goles enc.' in table.text.strip() or 'Asist.' in table.text.strip():
                    aux.append(table)

            tablas = [aux[0],aux[1],aux[4]]

            for tabla in tablas:
                filas = tabla.find_all('tr')
                nombre = filas[1].find_all('td')[0].text.strip()
                img = filas[1].find_all('td')[0].find_all('a')[1].attrs['href']
                enlace = 'https://es.wikipedia.org' + img
                numero = int(filas[1].find_all('td')[2].text.strip())
                resultado.append((nombre,[numero,enlace]))

                cont = 0

            print('\n')
            print('RESULTADOS')
            print('---------------------------------------------------------')
            for i in range(len(resultado)):
                if cont == 0:
                    k,v = resultado[i]
                    cont += 1
                    print(f'Máximo goleador: {k}')
                    print(f'Goles: {v[0]}')
                    img_wiki(v[1])
                    print('---------------------------------------------------------')
                elif cont == 1:
                    k,v = resultado[i]
                    cont += 1
                    print(f'Máximo asistente: {k}')
                    print(f'Asistencias: {v[0]}')
                    img_wiki(v[1])
                    print('---------------------------------------------------------')
                else:
                    k,v = resultado[i]
                    print(f'Mejor portero: {k}')
                    print(f'Goles encajados: {v[0]}')
                    img_wiki(v[1])
                    print('---------------------------------------------------------')

        elif menu == '3':
            url = 'https://es.wikipedia.org/wiki/Primera_Divisi%C3%B3n_de_Espa%C3%B1a_2012-13'
            result = requests.get(url)

            src = result.content
            soup = BeautifulSoup(src, 'html.parser')

            resultado = []

            aux = []
            tables = soup.find_all('table', class_='wikitable')
            for table in tables:
                if 'Goles' in table.text.strip() or 'Goles enc.' in table.text.strip() or 'Asist.' in table.text.strip():
                    aux.append(table)

            tablas = [aux[0],aux[1],aux[4]]

            for tabla in tablas:
                filas = tabla.find_all('tr')
                nombre = filas[1].find_all('td')[0].text.strip()
                img = filas[1].find_all('td')[0].find_all('a')[1].attrs['href']
                enlace = 'https://es.wikipedia.org' + img
                numero = int(filas[1].find_all('td')[2].text.strip())
                resultado.append((nombre,[numero,enlace]))

            cont = 0

            print('\n')
            print('RESULTADOS')
            print('---------------------------------------------------------')
            for i in range(len(resultado)):
                if cont == 0:
                    k,v = resultado[i]
                    cont += 1
                    print(f'Máximo goleador: {k}')
                    print(f'Goles: {v[0]}')
                    img_wiki(v[1])
                    print('---------------------------------------------------------')
                elif cont == 1:
                    k,v = resultado[i]
                    cont += 1
                    print(f'Máximo asistente: {k}')
                    print(f'Asistencias: {v[0]}')
                    img_wiki(v[1])
                    print('---------------------------------------------------------')
                else:
                    k,v = resultado[i]
                    print(f'Mejor portero: {k}')
                    print(f'Goles encajados: {v[0]}')
                    img_wiki(v[1])
                    print('---------------------------------------------------------')

        elif menu == '4':
            url = 'https://es.wikipedia.org/wiki/Primera_Divisi%C3%B3n_de_Espa%C3%B1a_2013-14'
            result = requests.get(url)

            src = result.content
            soup = BeautifulSoup(src, 'html.parser')

            resultado = []

            tables = soup.find_all('table', align='border', width='71%', border='1', cellpadding='2', cellspacing='0', style='background: #f9f9f9; border: 1px #aaa solid; border-collapse: collapse; font-size: 90%; text-align: center;')
            tablas = [tables[0],tables[1],tables[4]]

            for tabla in tablas:
                filas = tabla.find_all('tr')
                nombre = filas[1].find_all('td')[0].text.strip()
                img = filas[1].find_all('td')[0].find_all('a')[1].attrs['href']
                enlace = 'https://es.wikipedia.org' + img
                numero = int(filas[1].find_all('td')[2].text.strip())
                resultado.append((nombre,[numero,enlace]))

            cont = 0

            print('\n')
            print('RESULTADOS')
            print('---------------------------------------------------------')
            for i in range(len(resultado)):
                if cont == 0:
                    k,v = resultado[i]
                    cont += 1
                    print(f'Máximo goleador: {k}')
                    print(f'Goles: {v[0]}')
                    img_wiki(v[1])
                    print('---------------------------------------------------------')
                elif cont == 1:
                    k,v = resultado[i]
                    cont += 1
                    print(f'Máximo asistente: {k}')
                    print(f'Asistencias: {v[0]}')
                    img_wiki(v[1])
                    print('---------------------------------------------------------')
                else:
                    k,v = resultado[i]
                    print(f'Mejor portero: {k}')
                    print(f'Goles encajados: {v[0]}')
                    img_wiki(v[1])
                    print('---------------------------------------------------------')

        elif menu == '5':
            url = 'https://es.wikipedia.org/wiki/Primera_Divisi%C3%B3n_de_Espa%C3%B1a_2014-15'
            result = requests.get(url)

            src = result.content
            soup = BeautifulSoup(src, 'html.parser')

            resultado = []

            tabla_gol = soup.find('table', class_='sortable', border='1', width='75%', cellpadding='2', cellspacing='0', style='background: #f9f9f9; border: 1px #aaa solid; border-collapse: collapse; font-size: 85%; text-align: left;')
            tabla_asist = soup.find('table', align='center', cellpadding='2', cellspacing='0', style='background: #f5faff; border: 1px #aaa solid; border-collapse: collapse; font-size: 90%;', width='50%')
            tabla_zamora = soup.find_all('table', align='border', width='75%', border='1', cellpadding='2', cellspacing='0', style='background: #f9f9f9; border: 1px #aaa solid; border-collapse: collapse; font-size: 90%; text-align: center;')[8]

            tablas = [tabla_gol, tabla_asist, tabla_zamora]

            for tabla in tablas:
                filas = tabla.find_all('tr')
                nombre = filas[1].find_all('td')[0].text.strip()
                img = filas[1].find_all('td')[0].find_all('a')[1].attrs['href']
                enlace = 'https://es.wikipedia.org' + img
                numero = int(filas[1].find_all('td')[2].text.strip())
                resultado.append((nombre,[numero,enlace]))

            cont = 0

            print('\n')
            print('RESULTADOS')
            print('---------------------------------------------------------')
            for i in range(len(resultado)):
                if cont == 0:
                    k,v = resultado[i]
                    cont += 1
                    print(f'Máximo goleador: {k}')
                    print(f'Goles: {v[0]}')
                    img_wiki(v[1])
                    print('---------------------------------------------------------')
                elif cont == 1:
                    k,v = resultado[i]
                    cont += 1
                    print(f'Máximo asistente: {k}')
                    print(f'Asistencias: {v[0]}')
                    img_wiki(v[1])
                    print('---------------------------------------------------------')
                else:
                    k,v = resultado[i]
                    print(f'Mejor portero: {k}')
                    print(f'Goles encajados: {v[0]}')
                    img_wiki(v[1])
                    print('---------------------------------------------------------')

        elif menu == '6':
            url = 'https://es.wikipedia.org/wiki/Primera_Divisi%C3%B3n_de_Espa%C3%B1a_2015-16'
            result = requests.get(url)

            src = result.content
            soup = BeautifulSoup(src, 'html.parser')

            resultado = []

            tabla_gol = soup.find_all('table', cellspacing='0', style='background: #E6EEE6; border: 1px #aaa solid; border-collapse: collapse; font-size: 85%;', width='100%')[0]
            tabla_asist = soup.find('table', align='center', cellpadding='2', cellspacing='0', style='background: #f9f9f9; border: 1px #aaa solid; border-collapse: collapse; font-size: 90%;', width='50%')
            tabla_zamora = soup.find_all('table', align='border', width='75%', border='1', cellpadding='2', cellspacing='0', style='background: #f9f9f9; border: 1px #aaa solid; border-collapse: collapse; font-size: 90%; text-align: center;')[2]

            tablas = [tabla_gol, tabla_asist, tabla_zamora]

            filas = tabla_gol.find_all('tr')
            nombre = filas[1].find_all('td')[0].text.strip()
            img = filas[1].find_all('td')[0].find_all('a')[1].attrs['href']
            enlace = 'https://es.wikipedia.org' + img
            numero = int(filas[1].find_all('td')[1].text.strip())
            resultado.append((nombre,[numero,enlace]))

            for tabla in tablas[1:]:
                filas = tabla.find_all('tr')
                nombre = filas[1].find_all('td')[0].text.strip()
                img = filas[1].find_all('td')[0].find_all('a')[1].attrs['href']
                enlace = 'https://es.wikipedia.org' + img
                numero = int(filas[1].find_all('td')[2].text.strip())
                resultado.append((nombre,[numero,enlace]))

            cont = 0

            print('\n')
            print('RESULTADOS')
            print('---------------------------------------------------------')
            for i in range(len(resultado)):
                if cont == 0:
                    k,v = resultado[i]
                    cont += 1
                    print(f'Máximo goleador: {k}')
                    print(f'Goles: {v[0]}')
                    img_wiki(v[1])
                    print('---------------------------------------------------------')
                elif cont == 1:
                    k,v = resultado[i]
                    cont += 1
                    print(f'Máximo asistente: {k}')
                    print(f'Asistencias: {v[0]}')
                    img_wiki(v[1])
                    print('---------------------------------------------------------')
                else:
                    k,v = resultado[i]
                    print(f'Mejor portero: {k}')
                    print(f'Goles encajados: {v[0]}')
                    img_wiki(v[1])
                    print('---------------------------------------------------------')

        elif menu == '7':
            url = 'https://es.wikipedia.org/wiki/Primera_Divisi%C3%B3n_de_Espa%C3%B1a_2016-17'
            result = requests.get(url)

            src = result.content
            soup = BeautifulSoup(src, 'html.parser')

            resultado = []

            tables = soup.find_all('table', cellspacing='0', style='background: #E6EEE6; border: 1px #aaa solid; border-collapse: collapse; font-size: 85%;', width='100%')
            tabla_zamora = soup.find_all('table', align='border', width='75%', border='1', cellpadding='2', cellspacing='0', style='background: #f9f9f9; border: 1px #aaa solid; border-collapse: collapse; font-size: 90%; text-align: center;')[2]

            tablas = [tables[0],tables[1],tabla_zamora]

            for tabla in tablas[:2]:
                filas = tabla.find_all('tr')
                nombre = filas[1].find_all('td')[0].text.strip()
                img = filas[1].find_all('td')[0].find_all('a')[1].attrs['href']
                enlace = 'https://es.wikipedia.org' + img
                numero = int(filas[1].find_all('td')[1].text.strip())
                resultado.append((nombre,[numero,enlace]))

            filas = tablas[2].find_all('tr')
            nombre = filas[1].find_all('td')[1].text.strip()
            img = filas[1].find_all('td')[1].find_all('a')[1].attrs['href']
            enlace = 'https://es.wikipedia.org' + img
            numero = int(filas[1].find_all('td')[3].text.strip())
            resultado.append((nombre,[numero,enlace]))

            cont = 0

            print('\n')
            print('RESULTADOS')
            print('---------------------------------------------------------')
            for i in range(len(resultado)):
                if cont == 0:
                    k,v = resultado[i]
                    cont += 1
                    print(f'Máximo goleador: {k}')
                    print(f'Goles: {v[0]}')
                    img_wiki(v[1])
                    print('---------------------------------------------------------')
                elif cont == 1:
                    k,v = resultado[i]
                    cont += 1
                    print(f'Máximo asistente: {k}')
                    print(f'Asistencias: {v[0]}')
                    img_wiki(v[1])
                    print('---------------------------------------------------------')
                else:
                    k,v = resultado[i]
                    print(f'Mejor portero: {k}')
                    print(f'Goles encajados: {v[0]}')
                    img_wiki(v[1])
                    print('---------------------------------------------------------')

        elif menu == '8':
            url = 'https://es.wikipedia.org/wiki/Primera_Divisi%C3%B3n_de_Espa%C3%B1a_2017-18'
            result = requests.get(url)

            src = result.content
            soup = BeautifulSoup(src, 'html.parser')

            resultado = []

            tablas = soup.find_all('table', cellspacing='0', style='background: #E6EEE6; border: 1px #aaa solid; border-collapse: collapse; font-size: 85%;', width='100%')[:3]

            filas = tablas[0].find_all('tr')
            nombre = filas[1].find_all('td')[0].text.strip()
            img = filas[1].find_all('td')[0].find_all('a')[1].attrs['href']
            enlace = 'https://es.wikipedia.org' + img
            numero = int(filas[1].find_all('td')[1].text.strip())
            resultado.append((nombre,[numero,enlace]))

            for tabla in tablas[1:]:
                filas = tabla.find_all('tr')
                nombre = filas[1].find_all('td')[1].text.strip()
                img = filas[1].find_all('td')[1].find_all('a')[1].attrs['href']
                enlace = 'https://es.wikipedia.org' + img
                numero = int(filas[1].find_all('td')[2].text.strip())
                resultado.append((nombre,[numero,enlace]))

            cont = 0

            print('\n')
            print('RESULTADOS')
            print('---------------------------------------------------------')
            for i in range(0,len(resultado),2):
                if cont == 0:
                    k1,v1 = resultado[i]
                    k2,v2 = resultado[i+1]
                    cont += 1
                    print(f'Máximo goleador y asistente: {k1}')
                    print(f'Goles/Asistencias: {v1[0]}/{v2[0]}')
                    img_wiki(v1[1])
                    print('---------------------------------------------------------')
                else:
                    k,v = resultado[i]
                    print(f'Mejor portero: {k}')
                    print(f'Goles encajados: {v[0]}')
                    img_wiki(v[1])
                    print('---------------------------------------------------------')

        elif menu == '9':
            url = 'https://es.wikipedia.org/wiki/Primera_Divisi%C3%B3n_de_Espa%C3%B1a_2018-19'
            result = requests.get(url)

            src = result.content
            soup = BeautifulSoup(src, 'html.parser')

            resultado = []

            tablas = soup.find_all('table', class_='wikitable sortable', width='100%', style='text-align:center;background: #E6EEE6; border: 1px; border-collapse: collapse; font-size: 85%;')

            filas = tablas[0].find_all('tr')
            nombre = filas[1].find_all('td')[1].text.strip()
            img = filas[1].find_all('td')[1].find_all('a')[1].attrs['href']
            enlace = 'https://es.wikipedia.org' + img
            numero = int(filas[1].find_all('td')[2].text.strip())
            resultado.append((nombre,[numero,enlace]))

            url_as = 'https://resultados.as.com/resultados/futbol/primera/2018_2019/ranking/jugadores/asistencias_gol/'
            result_as = requests.get(url_as)
            soup_as = BeautifulSoup(result_as.content, 'html.parser')
            tabla_as = soup_as.find('table')
            columna_nom = tabla_as.find_all('tr')[1].find_all('td')[1]
            nombre = columna_nom.find('span', class_='name').text.strip()
            enlace_img = 'https:' + columna_nom.find('img').attrs['src']
            numero = tabla_as.find_all('tr')[1].find_all('td')[3].text.strip()[:2]
            resultado.append((nombre,[numero,enlace_img]))

            filas = tablas[1].find_all('tr')
            nombre = filas[1].find_all('td')[1].text.strip()
            img = filas[1].find_all('td')[1].find_all('a')[1].attrs['href']
            enlace = 'https://es.wikipedia.org' + img
            numero = int(filas[1].find_all('td')[3].text.strip())
            resultado.append((nombre,[numero,enlace]))

            cont = 0

            print('\n')
            print('RESULTADOS')
            print('---------------------------------------------------------')
            for i in range(len(resultado)):
                if cont == 0:
                    k,v = resultado[i]
                    cont += 1
                    print(f'Máximo goleador: {k}')
                    print(f'Goles: {v[0]}')
                    img_wiki(v[1])
                    print('---------------------------------------------------------')
                elif cont == 1:
                    k,v = resultado[i]
                    cont += 1
                    print(f'Máximo asistente: {k}')
                    print(f'Asistencias: {v[0]}')
                    img_AS(v[1])
                    print('---------------------------------------------------------')
                else:
                    k,v = resultado[i]
                    print(f'Mejor portero: {k}')
                    print(f'Goles encajados: {v[0]}')
                    img_wiki(v[1])
                    print('---------------------------------------------------------')

        elif menu == '10' or menu == '11':
            url = 'https://es.wikipedia.org/wiki/Primera_Divisi%C3%B3n_de_Espa%C3%B1a_20' + temporada[menu]
            result = requests.get(url)

            src = result.content
            soup = BeautifulSoup(src, 'html.parser')

            resultado = []

            tablas = soup.find_all('table', class_='wikitable sortable', width='100%', style='text-align:center;background: #E6EEE6; border: 1px; border-collapse: collapse; font-size: 85%;')
            for tabla in tablas[:2]:
                filas = tabla.find_all('tr')
                nombre = filas[1].find_all('td')[1].text.strip()
                img = filas[1].find_all('td')[1].find_all('a')[1].attrs['href']
                enlace = 'https://es.wikipedia.org' + img
                numero = int(filas[1].find_all('td')[2].text.strip())
                resultado.append((nombre,[numero,enlace]))

            filas = tablas[2].find_all('tr')
            nombre = filas[1].find_all('td')[1].text.strip()
            img = filas[1].find_all('td')[1].find_all('a')[1].attrs['href']
            enlace = 'https://es.wikipedia.org' + img
            numero = int(filas[1].find_all('td')[3].text.strip())
            resultado.append((nombre,[numero,enlace]))

            cont = 0

            print('\n')
            print('RESULTADOS')
            print('---------------------------------------------------------')
            if resultado[0][0] != resultado[1][0]:
                for i in range(len(resultado)):
                    if cont == 0:
                        k,v = resultado[i]
                        cont += 1
                        print(f'Máximo goleador: {k}')
                        print(f'Goles: {v[0]}')
                        img_wiki(v[1])
                        print('---------------------------------------------------------')
                    elif cont == 1:
                        k,v = resultado[i]
                        cont += 1
                        print(f'Máximo asistente: {k}')
                        print(f'Asistencias: {v[0]}')
                        img_wiki(v[1])
                        print('---------------------------------------------------------')
                    else:
                        k,v = resultado[i]
                        print(f'Mejor portero: {k}')
                        print(f'Goles encajados: {v[0]}')
                        img_wiki(v[1])
                        print('---------------------------------------------------------')
            else:
                for i in range(0,len(resultado),2):
                    if cont == 0:
                        k1,v1 = resultado[i]
                        k2,v2 = resultado[i+1]
                        cont += 1
                        print(f'Máximo goleador y asistente: {k1}')
                        print(f'Goles/Asistencias: {v1[0]}/{v2[0]}')
                        img_wiki(v1[1])
                        print('---------------------------------------------------------')
                    else:
                        k,v = resultado[i]
                        print(f'Mejor portero: {k}')
                        print(f'Goles encajados: {v[0]}')
                        img_wiki(v[1])
                        print('---------------------------------------------------------')

        menu = input('Temporadas: \n 1. 2010-11 \n 2. 2011-12 \n 3. 2012-13 \n 4. 2013-14 \n 5. 2014-15 \n 6. 2015-16 \n 7. 2016-17 \n 8. 2017-18 \n 9. 2018-19 \n 10. 2019-20 \n 11. 2020-21 \n 0. Salir \n\n Opción: ')

# 5. Asistencia media al campo de un equipo

In [7]:
def equipos_market(temp):
    headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.3"}
    #To make the request to the page we have to inform the
    #website that we are a browser and that is why we
    #use the headers variable
    link = f'https://www.transfermarkt.es/laliga/besucherzahlen/wettbewerb/ES1/plus/1?saison_id={temp}'
    objeto_response = requests.get(link, headers=headers)
    pagina = BeautifulSoup(objeto_response.content, 'html.parser')
    tabla = pagina.find('table', class_='items')
    filas = tabla.find_all('tr')
    nombres = filas[4::3]
    nombres = [str(x.text) for x in nombres]
    return (nombres, filas)



def equipos(temp): #Saca la lista de equipos de una temporada
    equipos = []
    link = f'https://es.wikipedia.org/wiki/Primera_Divisi%C3%B3n_de_Espa%C3%B1a_{temp}'
    src = requests.get(link)
    soup = BeautifulSoup(src.content, "html")
    tablas = soup.find_all('table', class_='sortable')
    for x in tablas:
        y = x.find('tr')
        z = y.find('th')
        if 'Equipo' in z.text:
            tabla = x
            break
    filas = tabla.find_all('tr')
    for fila in range(1, len(filas)):
        celdas = filas[fila].find_all('td')
        try:
            equipos.append(celdas[0].text.splitlines()[0])
        except IndexError:
            pass
    return equipos



def preguntar(algo):
    print('COD    AÑO')
    for ind,elm in enumerate(algo): #El usuario ha de ver las opciones
        print(ind,'-->',elm)
    a = False
    while a == False:
        try:
            r1 = int(input('COD:'))
            a = True
            if r1 < 0 or r1 > len(algo)-1:
                print('El COD que has seleccionado no pertenece a ninguna opción')
                a = False
        except ValueError:
            print('Error! Tienes que escribir un número')
            a = False
        except IndexError:
            print('Vuelve a escribirlo')
            a = False
    return r1




def asistencia():
    temps = [f'200{x}' for x in range(5,9)] + [f'201{x}' for x in range(0,9)] + ['2019'] + ['2020']
    año = preguntar(temps)
    teams, filas = equipos_market(temps[año])
    r1 = preguntar(teams)
    todo_nom = filas[2::3]
    asis = todo_nom[r1].find_all('td')
    estadio = asis[3].text
    cap = asis[5].text
    media = asis[7].text
    print(f'En la temporada de {temps[año]} el equipo {teams[r1]} contó con:')
    print(f'Una media de {media} espectadores teniendo una capacidad máxima de {cap} en su estadio {estadio}')
    

# 6. Resultados de un equipo según el árbitro

In [5]:
def res_arbitros():    
    victorias=0
    empates=0
    derrotas=0
    golesfav=0
    golescont=0


    listaarbs=[]

    listabuena=[]

    def arbfunc(ini,fin):
        for palabra in fin:
            if palabra not in ini:
                return False

        return True
    for temporada in range(2016,2021):


        url=f'https://es.wikipedia.org/wiki/Primera_Divisi%C3%B3n_de_Espa%C3%B1a_{temporada}-{temporada-1999}'
        listas=['font-size:95%; text-align: center; width: 100%;',"font-size:95%; text-align: center; width: 100%; margin:0.5em auto 0.5em auto;"]
        result=requests.get(url)
        src=result.content

        soup=BeautifulSoup(src,'lxml')

        arb=soup.find('div',class_='columns')
        arb2=arb.find_all('li')
        for arbitro in arb2:
            a=arbitro.find('small')
            b=a.find('a')
            if b.text not in listaarbs:
                listaarbs.append(b.text)
    for i,n in enumerate(listaarbs):
        print(i,')',n)
    arb=int(input('Árbitro: '))
    arbitro=listaarbs[arb]
    liy=arbitro.split()
    listaequips=[]    
    for temp in range(2016,2021):
        url=f'https://es.wikipedia.org/wiki/Primera_Divisi%C3%B3n_de_Espa%C3%B1a_{temp}-{temp-1999}'
        listas=['font-size:95%; text-align: center; width: 100%;',"font-size:95%; text-align: center; width: 100%; margin:0.5em auto 0.5em auto;"]
        result=requests.get(url)
        src=result.content

        soup=BeautifulSoup(src,'lxml')
        tabla=soup.find('table',cellpadding='3', align='center')
        equipos=tabla.find_all('tr')
        del equipos[0]
        listaequipos=[]
        for equipo in(equipos):
            datos=equipo.find_all('td')
            del datos[-1]
            for dato in datos:
                ex=dato.find('a')
                if ex is not None:
                    listaequipos.append(ex)
        listabuena=[]
        for e in listaequipos:
            if len(e.text)==0:
                pass
            else:

                listaequips.append(e.text)
    listaeqps=[]
    for i in range(60):
        if listaequips[i] not in listaeqps:
            listaeqps.append(listaequips[i])
    for i,e in enumerate(listaeqps):
        print(i,')',e)
    eq=int(input('Equipo: '))
    equipo=listaeqps[eq]
    ew=equipo.split()
    partidostot=[]
    tr=0
    for tempo in range(2016,2021):
        if tempo==2020:
            tr=1
        url=f'https://es.wikipedia.org/wiki/Primera_Divisi%C3%B3n_de_Espa%C3%B1a_{tempo}-{tempo-1999}'
        listas=['font-size:95%; text-align: center; width: 100%;',"font-size:95%; text-align: center; width: 100%; margin:0.5em auto 0.5em auto;"]
        result=requests.get(url)
        src=result.content

        soup=BeautifulSoup(src,'lxml')

        partidos2=[]
        partidos3=[]

        jornadas=soup.find_all('table',style=listas[tr])
        for jornada in jornadas:
            partidos=[]
            part=jornada.find_all('tr')
            if len(part)==12:
                partidos2.append(part.copy())
        for jornada in partidos2:
            for i in range(2):
                del jornada[0]
            for partido in jornada:
                datos=partido.find_all('td')
                listadatos=[]
                for dato in datos:
                    listadatos.append(dato.text)
                partidos3.append(listadatos)
        for partido in partidos3:
            del partido[3]
            for i in range(3):
                if len(partido)>6:
                    del partido[3]

        partidostot.append(partidos3)
    listapartidoscasa=[]
    listapartidosfuera=[]
    for temporada in partidostot:
        for partido in temporada:
            a=partido[0]
            b=partido[2]
            c=partido[3]
            if ((arbfunc(a.split(),ew) or arbfunc(ew,a.split()))) and (arbfunc(liy,c.split())):

                listapartidoscasa.append(partido)
            elif ((arbfunc(b.split(),ew) or arbfunc(ew,b.split()))) and (arbfunc(liy,c.split())):
                listapartidosfuera.append(partido)
    for part in listapartidoscasa:
        resul=part[1].split()
        golesfav+=int(resul[0])
        golescont+=int(resul[2])
        if int(resul[0])>int(resul[2]):
            victorias+=1
        elif int(resul[0])==(resul[2]):
            empates+=1
        else:
            derrotas+=1
    for part in listapartidosfuera:
        resul=part[1].split()
        golesfav+=int(resul[2])
        golescont+=int(resul[0])
        if int(resul[0])>int(resul[2]):
            derrotas+=1
        elif int(resul[0])==int(resul[2]):
            empates+=1
        else:
            victorias+=1
    print('Victorias: ',victorias)
    print('Derrotas: ',derrotas)
    print('Empates ',empates)
    print('Goles a favor: ', golesfav)
    print('Goles en contra: ',golescont)

# 7. Balance de goles por temporada

In [6]:
def plot_goles(t, r):
    leyenda = ['Goles a favor', 'Goles en contra']
    numeros = [[],[]]
    for gf,gc in r:
        numeros[0].append(gf)
        numeros[1].append(gc)
    for elem in numeros:
        plt.plot(t,elem)
    plt.xlabel("Temporada")             
    plt.ylabel("Cantidad")
    plt.legend(leyenda)
    plt.grid()
    plt.show()

def balance():
    clubes = {'1':'Real Madrid Club de Fútbol','2':'Fútbol Club Barcelona', '3':'Club Atlético de Madrid', '4':'Sevilla Fútbol Club', '5':'Valencia Club de Fútbol', '6':'Athletic Club', '7':'Real Sociedad de Fútbol'}
    temporadas = ['10-11','11-12','12-13','13-14','14-15','15-16','16-17','17-18','18-19','19-20','20-21']

    menu = input('Equipos: \n 1. Real Madrid Club de Fútbol \n 2. Fútbol Club Barcelona \n 3. Club Atlético de Madrid \n 4. Sevilla Fútbol Club \n 5. Valencia Club de Fútbol \n 6. Athletic Club \n 7. Real Sociedad de Fútbol \n 0. Salir \n\n Opción: ')

    while menu != '0':
        if all(menu != str(i) for i in range(0,8)):
            print(' Error. Por favor, elija una opción adecuada.')
        else:    
            resultado = []

            for i in temporadas:
                url = 'https://es.wikipedia.org/wiki/Primera_Divisi%C3%B3n_de_Espa%C3%B1a_20' + i
                result = requests.get(url)
                soup = BeautifulSoup(result.content, 'html.parser')
                tabla = soup.find('table', align='center', cellpadding='3', cellspacing='0', style='background: #F5F5F5; text-align:center; border-collapse: collapse; font-size: 90%;')
                filas = tabla.find_all('tr')[1:]
                for fila in filas:
                    if clubes[menu] == fila.find('a').attrs['title']:
                        gf = int(fila.find_all('td')[6].text.strip())
                        gc = int(fila.find_all('td')[7].text.strip())
                        resultado.append((gf,gc))
                        break

            print('---------------------------------------------------------')        
            print('RESULTADOS')
            for dato in zip(temporadas,resultado):
                print(f'- Temporada 20{dato[0]}: {dato[1][0]}/{dato[1][1]} (GF/GC)')
            plot_goles(temporadas, resultado)
            print('---------------------------------------------------------')

        menu = input('Equipos: \n 1. Real Madrid Club de Fútbol \n 2. Fútbol Club Barcelona \n 3. Club Atlético de Madrid \n 4. Sevilla Fútbol Club \n 5. Valencia Club de Fútbol \n 6. Athletic Club \n 7. Real Sociedad de Fútbol \n 0. Salir \n\n Opción: ')

# 8. PROGRAMA PRINCIPAL

In [8]:
opciones = ['Clasificación de la Liga',
            'Estadísticas de Árbitros',
            'Enfrentamiento Histórico entre equipos',
            'Jugadores más destacados',
            'Asistencia media al campo de un equipo',
            'Resultados de un equipo según el árbitro',
            'Balance de goles por temporada'] 

print('COD --> OPCIÓN')
print('-'*50)
for i,v in enumerate(opciones):
    print(i, '-->', v)
    print('-'*50)

a = False
while a == False:
        try:
            r1 = int(input('COD:'))
            a = True
            if r1 < 0 or r1 > len(opciones)-1:
                print('El COD que has seleccionado no pertenece a ninguna opción')
                a = False
        except ValueError:
            print('Error! Tienes que escribir un número')
            a = False
        except IndexError:
            print('Vuelve a escribirlo')
            a = False
            
if r1 == 0:
    clasificacion()
elif r1 == 1:
    arbitros()
elif r1 == 2:
    historico()
elif r1 == 3:
    jugadores_destacados()
elif r1 == 4:
    asistencia()
elif r1 == 5:
    res_arbitros()
elif r1 == 6:
    balance()

COD --> OPCIÓN
--------------------------------------------------
0 --> Clasificación de la Liga
--------------------------------------------------
1 --> Estadísticas de Árbitros
--------------------------------------------------
2 --> Enfrentamiento Histórico entre equipos
--------------------------------------------------
3 --> Jugadores más destacados
--------------------------------------------------
4 --> Asistencia media al campo de un equipo
--------------------------------------------------
5 --> Resultados de un equipo según el árbitro
--------------------------------------------------
6 --> Balance de goles por temporada
--------------------------------------------------
COD:0
COD    AÑO
0 --> 2001-02
1 --> 2002-03
2 --> 2003-04
3 --> 2004-05
4 --> 2005-06
5 --> 2006-07
6 --> 2007-08
7 --> 2008-09
8 --> 2010-11
9 --> 2011-12
10 --> 2012-13
11 --> 2013-14
12 --> 2014-15
13 --> 2015-16
14 --> 2016-17
15 --> 2017-18
16 --> 2018-19
17 --> 2019-20
18 --> 2020-21
COD:10
Pos. | Equi